In [2]:
import urllib.request
from IPython import get_ipython

from IPython.display import display  # To display various types of objects

 
# Fixa felaktiga priser  
- under ett antal veckor sattes priserna felaktigt till streck  
- scraping av priser för alla lopp i de felaktiga omgångarna  
- byt it de felaktiga priserna mot de nya riktiga priserna  
- Jag fick problem med många threads så det fick bli 8 åt gången

Kör sedan om learning-delen och skapa nya modeller

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

import concurrent.futures

In [5]:
import time

Hitta datum med felaktiga pris

In [6]:
all_data = pd.read_csv('../all_data.csv')
datum_list = all_data.loc[all_data.pris<5000].datum.unique().tolist()
print(datum_list)

[]


In [128]:
def get_webdriver(headless=True):
    # instance of Options class allows
    # us to configure Headless Chrome
    options = Options()
    if headless:
        # this parameter tells Chrome that
        # it should be run without UI (Headless)
        options.headless = True
        
        # the following seems to be necessary for headless to work
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
        options.add_argument('user-agent={0}'.format(user_agent))
        options.add_argument("--window-size=1920x1080")
        
    # initializing webdriver for Chrome with our options

    driver = webdriver.Chrome(
        executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe', options=options)
    return driver


In [129]:
### Priser ###
# span.css-1lnkuf6-startlistraceinfodetails-styles--infoContainer
def get_priser(driver,datum):
    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.css-1lnkuf6-startlistraceinfodetails-styles--infoContainer')))
    driver.implicitly_wait(5)
    priselement = driver.find_elements(By.CSS_SELECTOR, 'span.css-1lnkuf6-startlistraceinfodetails-styles--infoContainer')
    
    assert len(priselement)>0, f'datum {datum}, Priser tomt'
    
    NOK, EUR = False, False
    for p in priselement:
        if 'EUR' in p.text:
            EUR = True
            break
        elif 'NOK' in p.text:
            NOK = True
            break
    print('EUR:', EUR, 'NOK:', NOK)
    # alla lopps priser
    priser = [p.text for p in priselement if 'Pris:' in p.text]
    if len(priser) == 0:
        priser = [p.text for p in priselement if 'åriga' not in p.text and (
            'EUR' in p.text or 'NOK' in p.text)]  # alla lopps priser
    print('ant priser', len(priser))
    
    assert len(priser) > 0, 'No priser found'
    
    uppdaterade_priser=[]
    for anr in range(len(priser)):
        if EUR:
            pris = priser[anr].split('-')[0] + '0'    # Euro -> SEK
            print(f'EUR to SEK: {pris}')
        elif NOK:
            pris = priser[anr].split('-')[0].replace('.', '')  # NOK -> SEK
            print(f'NRK to SEK: {pris}')
        else:
            assert len(priser[anr].split('-')[0].split(' ')) > 0, f'pris saknas för {anr}'
            pris = priser[anr].split('-')[0].split(' ')[1]  
            
        pris=pris.replace(' ', '')
        pris=pris.replace('.', '')     
        uppdaterade_priser.append(pris)
   
    print(uppdaterade_priser)
    return uppdaterade_priser


In [130]:
# priselement = driv.find_elements(
#         By.CSS_SELECTOR, 'span.css-1lnkuf6-startlistraceinfodetails-styles--infoContainer')

In [131]:
# priselement

In [132]:
def bygg_url_list(datum_list):
    url_list = []
    for dat in datum_list:
        url_list.append("https://www.atg.se/spel/"+ dat +"/V75/")
        
    return url_list

In [133]:
def handle_one_week(datum,driver,url):
    global dat
    global driv
    global Url
    
    dat=datum 
    driv=driver 
    Url=url
    # assert False, f"take a brek at {datum} with url {url}"
    print('handling', datum)
    driver.implicitly_wait(10)     # seconds
    driver.get(url)      # öppna startlista
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"onetrust-accept-btn-handler"))).click()
    driver.fullscreen_window()
    try :
        bg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "bg")))
        bg.find_elements(By.CLASS_NAME, "race-info-toggle")[0].click()  # öppna prissummor mm
    except:    
        print('Fel: datum',datum, 'url',url)
        # driver.find_elements(By.CLASS_NAME, "race-info-toggle")[1].click()   # försök igen
    
    driver.implicitly_wait(5)
    priser={}
    priser[datum] = get_priser(driver,datum)
    print('end handling datum',datum)
    return priser

In [134]:
# datum=dat
# driver=driv
# url=Url
# print('handling', datum)
# driver.implicitly_wait(5)     # seconds
# driver.get(url)      # öppna startlista


In [135]:
# WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"onetrust-accept-btn-handler"))).click()
# driver.fullscreen_window()


In [136]:
# try :
#     bg = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "bg")))
#     bg.find_elements(By.CLASS_NAME, "race-info-toggle")[0].click()  # öppna prissummor mm
# except:    
#     print('Fel: datum',datum, 'url',url)
#     # driver.find_elements(By.CLASS_NAME, "race-info-toggle")[1].click()   # försök igen
# priser={}
# priser[datum] = get_priser(driver)
# print('end handling datum',datum)


## Starta Här!
Öppnar v75 sidorna (startlist plus ev resultat) och samlar all info per avd/häst

In [137]:

### Hela loopen med alla lopp i alla veckor i omg_df ###
start_time = time.perf_counter()
print('len(datum_list)',len(datum_list))
n_fr = 16+8+8+8+8
n_to = len(datum_list)  # n_fr+8
datum_list = datum_list[n_fr:n_to]  # de n_fr-n_to mellersta veckorna

ant_veckor = len(datum_list)     
print('ant_veckor', ant_veckor, 'från', n_fr, 'till', n_to)
url_list = bygg_url_list(datum_list)

print('start get webdrivers')
with concurrent.futures.ThreadPoolExecutor(max_workers=ant_veckor) as executor:
    # map excutor  driver_list and get_webdriver
    driver_list = list(executor.map(get_webdriver, [True]*ant_veckor))
print('end get webdrivers')
    


len(datum_list) 49
ant_veckor 1 från 48 till 49
start get webdrivers


C:\Users\peter\AppData\Local\Temp/ipykernel_38004/1955018772.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


end get webdrivers


In [138]:

pris_dict= {}
# for enum, datum in enumerate(datum_list[:3]):
#     print(enum, url_list[enum], driver_list[enum])
#     pris_dict[datum]= handle_one_week(driver_list[enum], url_list[enum])
    
print(f'start executor')
assert len(datum_list) == len(url_list) == len(driver_list), 'len(datum_list)!= len(url_list)!= len(driver_list)'
with concurrent.futures.ThreadPoolExecutor(max_workers=ant_veckor) as executor:
    # map excutor driver_avd_tab with ett_lopp(i,driver_avd_tab[i], comb, priser, result_tab, vdict, datum)
    future_list = [executor.submit(handle_one_week, datum_list[i],driver_list[i], url_list[i]) for i in range(len(datum_list))]
print(f'end executor')

for enum,future in enumerate(concurrent.futures.as_completed(future_list)):
    # try:
        data = future.result()
        datum = list(data.keys())[0] # the first (and only) key in data
        assert len(datum_list) > enum, f'len(datum_list) {len(datum_list)} <= enum {enum}'
        print(f'future enum={enum} {data} len(data)={len(data)}')
        pris_dict[datum] = data[datum]
        print(f'datum gick bra {datum} enum={enum}')
    # except Exception as exc:
    #     print(f'generated an exception in {datum_list[enum]} enum={enum} :')
    #     print('%r in "en_vecka" generated an exception: %s' % (future, exc))
    #     break


start executor
handling 2022-03-12
EUR: False NOK: False
ant priser 7
['110000', '110000', '150000', '110000', '110000', '110000', '125000']
end handling datum 2022-03-12
end executor
future enum=0 {'2022-03-12': ['110000', '110000', '150000', '110000', '110000', '110000', '125000']} len(data)=1
datum gick bra 2022-03-12 enum=0


In [139]:
    
print('start quit drivers')   
with concurrent.futures.ThreadPoolExecutor(max_workers=ant_veckor) as executor:
    # map excutor  driver_list and quit
    executor.map(lambda driver: driver.quit(), driver_list)
print('end quit drivers')
print('\ndet tog', round(time.perf_counter() - start_time, 3), 'sekunder')

df_pris = pd.DataFrame(pris_dict)
df_pris.to_csv('df_pris7.csv')
df_pris


start quit drivers
end quit drivers

det tog 6.837 sekunder


,2022-03-12
0,110000
1,110000
2,150000
3,110000
4,110000
5,110000
6,125000


Bygg ihop alla df_prisx.csv till en df_pris_all.csv 

In [198]:
df_pris1 = pd.read_csv('df_pris1.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_pris2 = pd.read_csv('df_pris2.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_pris3 = pd.read_csv('df_pris3.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_pris4 = pd.read_csv('df_pris4.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_pris5 = pd.read_csv('df_pris5.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_pris6 = pd.read_csv('df_pris6.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_pris7 = pd.read_csv('df_pris7.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
# Bygg ihop alla df_pris till en df_pris_all
df_pris_all = pd.concat([df_pris1, df_pris2, df_pris3, df_pris4, df_pris5, df_pris6, df_pris7], axis=1)
df_pris_all = df_pris_all.T     # transpose
df_pris_all.sort_index(ascending=True,inplace=True)
# set name of index to datum
df_pris_all.index.name = 'datum'
display(df_pris_all)
df_pris_all.to_csv('df_pris_all.csv')

,0,1,2,3,4,5,6
datum,,,,,,,
2021-05-01,110000,125000,110000,110000,110000,150000,400000
2021-05-08,1000000,110000,1000000,110000,110000,250000,1500000
2021-05-15,110000,110000,200000,110000,125000,200000,110000
2021-05-22,125000,110000,110000,200000,110000,400000,110000
2021-05-29,220000,220000,250000,220000,220000,220000,1000000
2021-05-30,500000,250000,250000,150000,300000,300000,500000
2021-06-05,110000,110000,150000,110000,110000,250000,125000
2021-06-12,110000,220000,110000,110000,110000,125000,700000
2021-06-19,110000,110000,110000,110000,110000,125000,1000000


Rätta till de felaktiga priserna i all_data.csv

In [246]:
# läs in all_data 
all_data = pd.read_csv('../all_data.csv')
# läs in df_pris_all.csv med de rätta priserna
df_pris_all = pd.read_csv('df_pris_all.csv', index_col=0)

def byt_pris(nytt_pris,datum):
    print(datum)
    return 1
    
    
df_pris_all.columns=[1,2,3,4,5,6,7]
# select all rows in all_data with datum in df_all_pris.index and use iterrose to loop over the rows

for index,row in all_data.loc[all_data.datum.isin(df_pris_all.index),['datum','avd','pris']].iterrows():
    # print(row[['datum','avd','pris']])
    # print(df_pris_all.loc[row.datum,row.avd])
    all_data.loc[index,'pris'] = df_pris_all.loc[row.datum,row.avd]


for index,row in df.loc[all_data.datum.isin(df2.datum),['datum','avd','pris']].iterrows():
    df.loc[index,'pris'] = df2.loc[row.datum,row.avd]


### Stickprovskontroller  

In [301]:
stickprov =[
    ['2021-12-31',	100000,	100000, 100000,	100000,	100000,	100000,	537629],  
    ['2022-02-05',	250000,	220000, 300000,	220000,	220000,	300000,	220000],  
    ['2021-05-01',	110000,	125000, 110000,	110000,	110000,	150000,	400000],  
    ['2022-03-12',	110000,	110000, 150000,	110000,	110000,	110000,	125000],
    ['2021-10-23',	125000,	110000,	150000,	110000,	110000,	110000,	110000],
    ['2021-03-13',  125000,	110000,	110000,	110000,	110000,	110000,	150000],  # skall vara oförändrad
    ['2020-03-14',  110000,	110000,	150000,	110000,	110000,	125000,	110000],  # skall vara oförändrad
]
for datum,avd1,avd2,avd3,avd4,avd5,avd6,avd7 in stickprov:
    assert (all_data.loc[(all_data.datum == datum) & (all_data.avd == 1)].pris == avd1).all() , f'priserna är fel för {datum} avd {1}'   
    assert (all_data.loc[(all_data.datum == datum) & (all_data.avd == 2)].pris == avd2).all() , f'priserna är fel för {datum} avd {2}'   
    assert (all_data.loc[(all_data.datum == datum) & (all_data.avd == 3)].pris == avd3).all() , f'priserna är fel för {datum} avd {3}'
    assert (all_data.loc[(all_data.datum == datum) & (all_data.avd == 4)].pris == avd4).all() , f'priserna är fel för {datum} avd {4}'
    assert (all_data.loc[(all_data.datum == datum) & (all_data.avd == 5)].pris == avd5).all() , f'priserna är fel för {datum} avd {5}'
    assert (all_data.loc[(all_data.datum == datum) & (all_data.avd == 6)].pris == avd6).all() , f'priserna är fel för {datum} avd {6}'
    assert (all_data.loc[(all_data.datum == datum) & (all_data.avd == 7)].pris == avd7).all() , f'priserna är fel för {datum} avd {7}'


In [302]:
# spara rättad all_data till all_data.csv
all_data.to_csv('../all_data.csv', index=False)